In [37]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.utils import resample
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

### 1. Utiliza los conceptos aprendidos en los laboratorios de regresión y clasificación para encontrar el error estándar de los coeficientes de una regresión (lineal/logística) simple para los datasets de “Advertising” y “Default”.

## ADVERTISING

In [5]:
defa = pd.read_csv("Default.csv")
adv = pd.read_csv("Advertising.csv"

In [15]:
# Primero importo los datos
adv = pd.read_csv("Advertising.csv")

# Agrego el intercepto
X = adv[['TV', 'radio', 'newspaper']].values
X_design = np.column_stack((np.ones(len(X)), X))   # columna de 1s

y = adv['sales'].values.reshape(-1,1)

# Modelo
lr = LinearRegression()
lr.fit(X, y)

# Saco las betas
betas = np.concatenate([lr.intercept_, lr.coef_[0]])

# Predicción
y_pred = lr.predict(X)

# Obtengo error
RSS = np.sum((y - y_pred)**2)

# Grados de libertad
n = len(y)
p = 3   # Corresponden a cada columna
sigma2 = RSS / (n - p - 1)

# Matriz de varianza de los coeficientes
XtX_inv = np.linalg.inv(X_design.T @ X_design)
var_betas = sigma2 * XtX_in

# Errores estándar
se_betas = np.sqrt(np.diag(var_betas))

se_b0 = se_betas[0]
se_b1 = se_betas[1]
se_b2 = se_betas[2]
se_b3 = se_betas[3]

se_b0, se_b1, se_b2, se_b3


(0.3119082363217909,
 0.0013948968069749741,
 0.008611233967301948,
 0.005871009647086368)

## DEFAULT

In [19]:
# Datos
defa = pd.read_csv("Default.csv")

# Recuerda que aquí lo queremos saber la predicción de que si vaya a dejar de pagar (creo)
y = (defa['default'] == "Yes").astype(int).values
x = defa[['balance']].values

# Ajustar regresión logística
lgr = LogisticRegression()
lgr.fit(x, y)

# Sacamos las probabilidades considerando nuestro y
p = lgr.predict_proba(x)[:, 1]

# W matrix
W = np.diag(p * (1 - p))

# X con intercepto
X = np.column_stack((np.ones(len(x)), x))

# Matriz de varianza (Hessian invertida)
cov = np.linalg.inv(X.T @ W @ X)

# Errores estándar (raíz de la diagonal)
se = np.sqrt(np.diag(cov))

se_0 = se[0]   # intercepto
se_1 = se[1]   # balance

se_0, se_1


(0.3611685997151162, 0.00022037616946191625)

### 2. Utiliza bootstrap para simular 1000 remuestreos de esos datasets y calcula la media de los coeficientes obtenidos al aplicarle regresión a cada remuestreo. Calcula la desviación estándar.

### Advertising

In [23]:
# Betas que teníamos arriba (las reales)
betas_lr = np.concatenate([lr.intercept_, lr.coef_[0]])

# Aqui pongo lso errores pero la pura diagonal
se_classic_adv = np.sqrt(np.diag(var_betas))

# Bootstrap

B = 1000
boot_betas_adv = []

for i in range(B):
    sample = resample(adv)
    Xb = sample[['TV', 'radio', 'newspaper']].values
    yb = sample['sales'].values.reshape(-1,1)

    lr_b = LinearRegression()
    lr_b.fit(Xb, yb)

    betas_b = np.concatenate([lr_b.intercept_, lr_b.coef_[0]])
    boot_betas_adv.append(betas_b)

boot_betas_adv = np.array(boot_betas_adv)

boot_mean_adv = boot_betas_adv.mean(axis=0)
boot_std_adv = boot_betas_adv.std(axis=0)


# Comparación de los erores y media

tabla_adv = pd.DataFrame({
    "Coeficiente": ["Intercepto", "TV", "Radio", "Newspaper"],
    "SE Clásico": se_classic_adv,
    "SE Bootstrap": boot_std_adv,
    "Media Bootstrap": boot_mean_adv
})

tabla_adv

,Coeficiente,SE Clásico,SE Bootstrap,Media Bootstrap
0,Intercepto,0.311908,0.343429,2.962338
1,TV,0.001395,0.001838,0.045663
2,Radio,0.008611,0.010295,0.188257
3,Newspaper,0.005871,0.006430,-0.000884


## DEFAULT

In [24]:
# Diagonal de las betas
se_classic_def = np.sqrt(np.diag(cov))  

# Bootstrap Default

boot_betas_def = []

for i in range(1000):
    sample = resample(defa)
    Xb = sample[['balance']].values
    yb = (sample['default'] == "Yes").astype(int).values

    lgr_b = LogisticRegression(solver="lbfgs")
    lgr_b.fit(Xb, yb)

    b0 = lgr_b.intercept_[0]
    b1 = lgr_b.coef_[0][0]
    boot_betas_def.append([b0, b1])

boot_betas_def = np.array(boot_betas_def)

boot_mean_def = boot_betas_def.mean(axis=0)
boot_std_def = boot_betas_def.std(axis=0)

# Comparación errores y media

tabla_def = pd.DataFrame({
    "Coeficiente": ["Intercepto", "Balance"],
    "SE Clásico": se_classic_def,
    "SE Bootstrap": boot_std_def,
    "Media Bootstrap": boot_mean_def
})


tabla_def

,Coeficiente,SE Clásico,SE Bootstrap,Media Bootstrap
0,Intercepto,0.361169,0.364683,-10.674687
1,Balance,0.000220,0.000223,0.005511


### 3. Compara los resultados obtenidos con el método visto en los laboratorios contra los resultados obtenidos con bootstrap. ¿Por qué podría haber diferencias en los resultados?

Bueno primeramente indicar que tanto en Advertising como en Default podemos notar que los valores de los errores se acercan demasiado, indicando que se hizo un buen proceso y que justo el punto de esto es que se asemejen. Solamente observé un ligero aumento en Bootstrap, pero igual no es tanto.

Algunas de las razones por las diferencias en los resultados son:
Primeramente el método clásico, por así llamarlo considera que los errores tiene una distribución normal, existe varianza constante, la relación entre x y y es casi perfecta y a diferencia del una vez haciendo bootstrap es que no asume normalidad ni homocedasticidad y usa la variable real de los datos, por ello que son un poco mayores.


En el data set de Advertising también influye como que el modelo tenga ruido, porque en las 1000 simulaciones bootstrap hace que la variación del coeficiente se aumente y que detecte esa inestabilidad.

Y en Default la verdad que los SE se parecían demasiado, y esto también es porque es un modelo simple y bien específicado y la relación entre balance y default es muy fuerte.

### 4. Agrega regularización L2 a los modelos del dataset de Advertising (optimiza el hiperparámetro). Utiliza ese valor del hiperparámetro para repetir el experimento de los 1000 remuestreos. Calcula la desviación estándar de los coeficientes obtenidos.

In [38]:
np.random.seed(14)

# --------------------------------------------------
# 1. Función objetivo: RMSE promedio para cada C
# --------------------------------------------------
def objective(C):
    ridge_clf = make_pipeline(
        StandardScaler(),
        Ridge(alpha=1/C)   # equivalente a C en logistic
    )
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(ridge_clf, X, y, cv=cv,
                             scoring='neg_root_mean_squared_error')
    return scores.mean()


X_C = np.random.uniform(0.01, 100, 3).reshape(-1, 1)
y_obj = np.array([objective(c[0]) for c in X_C]).reshape(-1, 1)

kernel = 1.0 * RBF(length_scale=1.0)
gp = GaussianProcessRegressor(kernel=kernel,
                              n_restarts_optimizer=10,
                              alpha=1e-6)

x_v = np.linspace(0.01, 100, 200).reshape(-1, 1)

for i in range(8): 
    gp.fit(X_C, y_obj)

    y_pred, std = gp.predict(x_v, return_std=True)
    idx_max = np.argmax(y_pred)
    C_best = x_v[idx_max][0]
    y_real = objective(C_best)
    X_C = np.vstack([X_C, [[C_best]]])
    y_obj = np.vstack([y_obj, [[y_real]]])


best_idx = np.argmax(y_obj)
C_best_final = X_C[best_idx][0]

print(f"Mejor C encontrado: {C_best_final:.4f}")

final_ridge = make_pipeline(
    StandardScaler(),
    Ridge(alpha=1/C_best_final)
)


n_boot = 1000
coefs_boot = []

for i in range(n_boot):
    sample_idx = np.random.choice(len(X), size=len(X), replace=True)
    X_res = X[sample_idx]
    y_res = y[sample_idx]

    final_ridge.fit(X_res, y_res)

    model = final_ridge.named_steps['ridge']
    coefs_boot.append(model.coef_)

coefs_boot = np.array(coefs_boot)

# Media y desviación bootstrap
mean_boot = coefs_boot.mean(axis=0)
std_boot = coefs_boot.std(axis=0)

print("\nMedia bootstrap de coeficientes:", mean_boot)
print("Desviación estándar bootstrap:", std_boot)


C:\Users\Admin\Documents\Programas\ANACONDA\N\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\Admin\Documents\Programas\ANACONDA\N\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\Admin\Documents\Programas\ANACONDA\N\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\Admin\D

Mejor C encontrado: 100.0000

Media bootstrap de coeficientes: [[ 3.91230888  2.7757363  -0.02268208]]
Desviación estándar bootstrap: [[0.22537808 0.18025908 0.13965729]]


Comparación sin la optimización

In [39]:
tabla_adv

,Coeficiente,SE Clásico,SE Bootstrap,Media Bootstrap
0,Intercepto,0.311908,0.343429,2.962338
1,TV,0.001395,0.001838,0.045663
2,Radio,0.008611,0.010295,0.188257
3,Newspaper,0.005871,0.006430,-0.000884


No pidió interpretación aquí pero igual quiero dejar mis dudas :)

No había entendido mucho sobre como realizar interpretaciones aquí, o bueno, no interpretaciones, sino el comparar la optimización con antes de hacerlo. Me confundió un poco porque no estábamos imprimiendo como tal los mismos valores, ya que antes nos encontrábamos con los errores y ahora con la media y la desviación.

Investigando un poco me doy cuenta que la media en bootstrap es como una estimiación de los coeficientes, por lo que puedo ver
que una vez haciendo la optimización los coeficientes aumentaron mucho más de lo que estaban antes, indicando que la regularización con Ridge al estandarizar las variables afecta sus coeficientes.
Y si comparamos un poco la desviación estándar con los errores veo que también aumentaron un poco. Creo que estos no se pueden comparar como tal, porque uno es en general y otro es más específico de una variable, pero igual pueden servir como guía.
Creo que algo hay al en esto de la optimización, porque me esperaba valores más congruentes o que me dijesen que la optimización ha valido la pena.